In [ ]:
### imports

# external modules
import os
import sys
import json
import glob
import tarfile as tar
import numpy as np
from fnmatch import fnmatch
import matplotlib.pyplot as plt
import importlib

# local modules
thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../'))
sys.path.append(topdir)

from automasking.tools.lumisection_time_parser import LumisectionTimeParser
from automasking.tools.automask_file_parsing import get_automask_from_tarfiles
from automasking.tools.automask_file_parsing import automask_to_map
import plotting.plottools as plottools

In [ ]:
# find and sort all tar files

# set input directory
inputdir = '/eos/project-m/mlplayground/public/PixelAutomasking/PixelAutoMasking/2024'

# find all .tar.xz files recursively
pattern = os.path.join(inputdir, '**/*.tar.xz')
tarfiles = glob.glob(pattern, recursive=True)
print(f'Found {len(tarfiles)} tar files in {inputdir}.')

# sort them according to date
tarfiles.sort(key=lambda x: os.path.basename(x))
for f in tarfiles[:10]: print(f'  - {f}')

In [ ]:
# make time parser instance

jsonfile = 'data/oms_data/omsdata_lumisection_timestamps_2024.json'
ltp = LumisectionTimeParser(jsonfile)

In [ ]:
# get automask

automask_info = get_automask_from_tarfiles(tarfiles, ltp=ltp)

In [ ]:
# write to json file

outputdir = 'data/automask_data'
if not os.path.exists(outputdir): os.makedirs(outputdir)
outputfile = os.path.join(outputdir, f'automask_2024.json')
with open(outputfile, 'w') as f:
    json.dump(automask_info, f)